![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=SocialStudies/HansardAnalysis/hansard-analysis.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Hansard Analysis

The [Hansard](https://en.wikipedia.org/wiki/Hansard) is a transcript of debates in the Canadian Parliament. It is available from the official [Parliament of Canada website](https://www.parl.ca) as well as other sources such as [Open Parliament](https://openparliament.ca) and [LiPaD: The Linked Parliamentary Data Project](https://www.lipad.ca/).

We have downloaded the 2020 files from LiPaD, and can load them by selecting the following code cell and clicking the `▶Run` button.

In [7]:
df = pd.read_csv('proceedings2020.csv')
print(df.shape)
df.columns

(4945, 15)


Index(['basepk', 'hid', 'speechdate', 'pid', 'opid', 'speakeroldname',
       'speakerposition', 'maintopic', 'subtopic', 'subsubtopic', 'speechtext',
       'speakerparty', 'speakerriding', 'speakername', 'speakerurl'],
      dtype='object')

There are 15 columns and 4945 rows of data. Let's have a look at who spoke during these debates.

In [14]:
speakers = df.drop_duplicates(subset=['speakername'])[['speakername','speakerparty','speakerriding','speakerurl']]
speakers = speakers.dropna().reset_index().drop(columns=['index'])
print('There were',speakers.shape[0],'speakers from the',speakers['speakerparty'].unique(),'parties.')

There were 312 speakers from the ['Liberal' 'New Democratic Party' 'Green Party' 'Conservative'
 'Bloc Québécois' 'Independent'] parties.


We can compare that to the list of Members of Parliament from the [43rd Parliament](https://en.wikipedia.org/wiki/43rd_Canadian_Parliament) that started on December 5, 2019.

In [20]:
import pandas as pd
members = pd.read_csv('https://www.ourcommons.ca/members/en/search/csv?parliament=43')
print('There were',members.shape[0],'Members from the',members['Political Affiliation'].unique(),'parties.')

There were 340 Members from the ['Conservative' 'Liberal' 'NDP' 'Green Party' 'Bloc Québécois'
 'Independent'] parties.


So of the 340 Members of Parliament we had 312 unique speakers, meaning that 28 Members are not recorded as speaking during 2020. Let's see if we can identify who are they were.

In [34]:
members['Name'] = members['First Name'] +' '+ members['Last Name']
silent = []
for member in members['Name']:
    if member not in speakers['speakername'].values:
        silent.append(member)
print('That is',len(silent),'Members not recorded as speaking in 2020.')
print(silent)

That is 35 Members not recorded as speaking in 2020.
['Jaime Battiste', 'Terry Beech', 'Bob Benzen', 'Stéphane Bergeron', 'Sylvie Bérubé', 'Yves-François Blanchet', 'Élisabeth Brière', 'Jim Carr', 'Sean Casey', 'Serge Cormier', 'Neil Ellis', 'Marie-Hélène Gaudreau', 'Marci Ien', 'Robert Kitchen', 'Andréanne Larouche', 'Sébastien Lemire', 'Dave MacKenzie', 'Simon Marcil', 'Ken McDonald', 'John McKay', 'Alexandra Mendès', 'Robert Morrissey', 'Joyce Murray', 'Robert Oliphant', "Seamus O'Regan", 'Louis Plamondon', 'Marcus Powlowski', 'Michelle Rempel Garner', 'Raj Saini', 'Judy A. Sgro', 'Terry Sheehan', 'Doug Shipley', 'Chris Warkentin', 'Alice Wong', 'Bill Morneau']


Of course 35 is not equal to 28, but we will leave it to you to compare the list `silent` to the list from `speakers['speakername'].unique()` if you are interested.

## Who Spoke Most?



## Natural Language Processing

In [36]:
df['speakerparty'].value_counts()

Liberal                 1939
Conservative            1625
New Democratic Party     531
Bloc Québécois           443
Green Party              125
Independent                8
Name: speakerparty, dtype: int64

In [12]:
px.bar(df['speakername'].value_counts().head(20))

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)